In [2]:
import napari
import numpy as np
import pandas as pd
from data import convert_dataframe_to_coords
from data import convert_tree_to_dataframe
from IPython.display import display
from skimage.io import imread
from skimage.measure import regionprops_table
from laptrack import LapTrack

In [3]:
images = imread("interactive_example_data/demo_image.tif")
labels = np.load("interactive_example_data/labels3.npy")

In [4]:
viewer = napari.Viewer()
viewer.add_image(images, name="images")
viewer.add_labels(labels, name="labels")

<Labels layer 'labels' at 0x1b40b3a60>

In [5]:
def calc_frame_regionprops(labels):  # noqa: E302
    dfs = []
    for frame in range(labels.shape[0]):
        df = pd.DataFrame(
            regionprops_table(labels[frame], properties=["label", "area", "centroid"])
        )
        df["frame"] = frame
        dfs.append(df)
    return pd.concat(dfs)
regionprops_df = calc_frame_regionprops(labels)
display(regionprops_df)

,label,area,centroid-0,centroid-1,frame
0,2,1149,21.095735,51.044386,0
1,3,676,13.699704,73.125740,0
2,4,900,13.463333,99.415556,0
3,5,706,14.866856,133.256374,0
4,6,542,9.046125,20.151292,0
...,...,...,...,...,...
34,36,905,65.306077,10.375691,9
35,37,110,14.627273,141.709091,9
36,38,741,33.570850,126.275304,9
37,39,158,133.164557,139.373418,9


In [6]:
_coords = convert_dataframe_to_coords(
    regionprops_df, ["centroid-0", "centroid-1", "label"]
)
coords = [c[:, :-1] for c in _coords]
coord_labels = [c[:, -1] for c in _coords]

In [7]:
lt = LapTrack(splitting_cost_cutoff=20**2)
tree = lt.predict(coords)
tracked_df, _ = convert_tree_to_dataframe(tree)

In [8]:
_regionprops_df = regionprops_df.set_index(["frame", "label"])
for (frame, index), row in tracked_df.iterrows():
    label = coord_labels[frame][index]
    _regionprops_df.loc[(frame, label), "cell_id"] = row["cell_id"] 
track_label_image = np.zeros_like(labels)
for (frame, label), row in _regionprops_df.iterrows():
    track_label_image[frame][labels[frame] == label] = row["cell_id"] + 1

In [9]:
viewer.layers["labels"].visible = False
viewer.add_labels(track_label_image)

<Labels layer 'track_label_image' at 0x1b92ffb80>

In [10]:
manual_corrected = np.load("interactive_example_data/manual_corrected.npy")
viewer.add_points(manual_corrected, name="manually_validated_tracks")

<Points layer 'manually_validated_tracks' at 0x1bf643d30>

In [11]:
manual_corrected = viewer.layers["manually_validated_tracks"].data.astype(np.int16)
# you can also redraw the labels
new_labels = viewer.layers["track_label_image"].data
# get label values at the placed points
validated_track_labels = new_labels[tuple(manual_corrected.T)]
validated_frames = manual_corrected[:, 0]

In [31]:
new_regionprops_df = calc_frame_regionprops(new_labels).set_index(["frame", "label"])
new_regionprops_df.loc[
    [(f, l) for f, l in zip(validated_frames, validated_track_labels)], "cell_id"
] = -1
new_regionprops_df["cell_id"] = new_regionprops_df["cell_id"].fillna(0).astype(np.int16)
new_regionprops_df = new_regionprops_df.reset_index()
display(new_regionprops_df[new_regionprops_df["cell_id"] == -1])

,frame,label,area,centroid-0,centroid-1,cell_id
33,0,34,621,23.745572,14.080515,-1
63,1,34,576,30.123264,14.315972,-1
95,2,37,537,26.126629,26.819367,-1
124,3,37,458,24.491266,31.026201,-1
158,4,37,336,25.056548,22.598214,-1
191,5,37,801,25.038702,30.917603,-1
223,6,37,382,29.615183,31.086387,-1
255,7,37,597,26.582915,33.195980,-1
290,8,37,561,26.584670,33.021390,-1
325,9,37,832,28.170673,33.709135,-1


In [32]:
_new_coords = convert_dataframe_to_coords(
    new_regionprops_df[new_regionprops_df["cell_id"] != -1],
    ["centroid-0", "centroid-1", "label"],
)
new_coords = [c[:, :-1] for c in _new_coords]
new_coord_labels = [c[:, -1] for c in _new_coords]

lt = LapTrack(splitting_cost_cutoff=20**2)
new_tree = lt.predict(new_coords)
new_tracked_df, _ = convert_tree_to_dataframe(new_tree)

In [33]:
_new_regionprops_df = new_regionprops_df.copy().set_index(["frame", "label"])
for (frame, index), row in new_tracked_df.iterrows():
    label = new_coord_labels[frame][index]
    _new_regionprops_df.loc[(frame, label), "cell_id"] = row["cell_id"] + 1
_new_regionprops_df.loc[_new_regionprops_df["cell_id"] == -1, "cell_id"] = (
    _new_regionprops_df["cell_id"].max() + 1
)

In [34]:
new_track_label_image = np.zeros_like(new_labels)
for (frame, label), row in _new_regionprops_df.iterrows():
    new_track_label_image[frame][new_labels[frame] == label] = row["cell_id"] + 1
viewer.layers["track_label_image"].visible = False
viewer.add_labels(new_track_label_image)

<Labels layer 'new_track_label_image' at 0x1c316dae0>